<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO 5: PROBLEMAS DERIVADOS DE LWE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Tras haber explicado y analizado el problema LWE, tanto en el documento como en código, vimos en el documento que de este nacen otros dos problemas, sustituyendo la primitiva matemática por otras más complejas como son los anillos y los módulos. En este anexo estudiaremos las implementaciones de los problemas derivados de LWE: RLWE y MLWE. 

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuración-previa)
3. [RLWE](#3.-RLWE)
4. [MLWE](#4.-MLWE)

---
# 1. Introducción






En este anexo se presentan las implementaciones prácticas de los problemas derivados del problema de Aprendizaje con errores (LWE): el problema Ring-LWE y Module-LWE.
Estos problemas que veremos implememtados a continuacion son los que realmente marcan la diferencia en Kyber-KEM, siendo los problemas en los que sustenta su seguridad.
A lo largo de este anexo, se detalla cómo se han implementado estos problemas, proporcionando un análisis del diseño de los algoritmos, los métodos de optimización utilizados y los resultados obtenidos en diferentes escenarios prácticos.

# 2. Configuración previa

In [3]:
#MODULOS A IMPORTAR
import numpy as np
import itertools
import random
from numpy.polynomial import Polynomial

---

# 3. RLWE

El problema de Learning With Errors (RLWE, por sus siglas en inglés) es un problema fundamentalmente difícil en la criptografía moderna. Está basado en la dificultad de resolver ciertos sistemas de ecuaciones polinomiales en cuerpos finitos, con un pequeño error añadido a los valores de las ecuaciones. Este problema es considerado un candidato sólido para la construcción de esquemas de cifrado resistentes a ataques cuánticos.

En el caso específico de RLWE, el objetivo es encontrar una clave secreta $s$, dada una instancia pública que consiste en dos polinomios $a$ y $b$, que están relacionados de la siguiente manera:

$$
b = a \cdot s + e
$$

donde $e$ es un pequeño error aleatorio y $s$ es la clave secreta.

La dificultad del problema reside en la imposibilidad de obtener $s$ de manera eficiente a partir de $a$ y $b$, debido a la presencia del error $e$.

<br>

El código implementa un esquema de cifrado basado en el problema de Learning With Errors (RLWE). Genera una instancia de este problema creando dos polinomios públicos $a$ y $b$, y una clave secreta $s$, que están relacionados por la ecuación $b = a \cdot s + e$, donde $e$ es un pequeño error aleatorio. Luego, utiliza esta instancia para cifrar y descifrar mensajes. El cifrado se realiza mediante la generación de dos componentes $u$ y $v$, que dependen de errores aleatorios y del mensaje. El descifrado recupera el mensaje original utilizando la clave secreta $s$ para despejar la información del cifrado. El proceso involucra operaciones de reducción módulo $q$ y la manipulación de polinomios en el anillo $\mathbb{Z}_q[x] / (x^n + 1)$.




In [4]:
def generate_secret_key(n, q):
    """
    Genera una clave secreta como un polinomio en Z_q[x]/(x^n + 1).

    Parámetros:
    - n: Grado del polinomio.
    - q: Módulo del polinomio.

    Retorna:
    - Polinomio secreto generado.
    """
    return Polynomial(np.random.randint(0, q, n))

def generate_error(n, error_level, q):
    """
    Genera un error pequeño basado en el nivel de ruido controlado.

    Parámetros:
    - n: Grado del polinomio.
    - error_level: Nivel de error (1-10).
    - q: Módulo del polinomio.

    Retorna:
    - Polinomio de error generado.
    """
    scaled_error = error_level / 5  # Escala el ruido de 0.2 a 2
    error = Polynomial(np.round(np.random.normal(0, scaled_error, n)).astype(int) % q)
    print(f"Error generado con nivel {error_level} (σ={scaled_error}): {error}")
    return error

def reduce_modulus(poly, q, n):
    """
    Reduce un polinomio módulo x^n + 1 en Z_q.

    Parámetros:
    - poly: Polinomio a reducir.
    - q: Módulo del polinomio.
    - n: Grado del polinomio.

    Retorna:
    - Polinomio reducido.
    """
    coeffs = np.mod(poly.coef[:n], q)
    return Polynomial(coeffs)

def generate_rlwe_instance(n, q, error_level):
    """
    Genera una instancia del problema RLWE: (a, b = a * s + e) en Z_q[x]/(x^n + 1).

    Parámetros:
    - n: Grado del polinomio.
    - q: Módulo del polinomio.
    - error_level: Nivel de error (1-10).

    Retorna:
    - a: Polinomio público 'a'.
    - b: Polinomio público 'b'.
    - s: Clave secreta 's'.
    """
    a = Polynomial(np.random.randint(0, q, n))
    s = generate_secret_key(n, q)
    e = generate_error(n, error_level, q)
    b = reduce_modulus(a * s + e, q, n)
    print(f"Ruido e en RLWE instancia con nivel {error_level}: {e}")
    return a, b, s

def rlwe_encrypt(m, a, b, q, n, error_level):
    """
    Cifra un mensaje en Z_q[x]/(x^n + 1) usando la instancia RLWE.

    Parámetros:
    - m: Mensaje a cifrar (vector de coeficientes).
    - a: Polinomio público 'a'.
    - b: Polinomio público 'b'.
    - q: Módulo del polinomio.
    - n: Grado del polinomio.
    - error_level: Nivel de error (1-10).

    Retorna:
    - u: Componente 'u' del cifrado.
    - v: Componente 'v' del cifrado.
    """
    e1 = generate_error(n, error_level, q)
    e2 = generate_error(n, error_level, q)
    print(f"Ruido e1 en cifrado con nivel {error_level}: {e1}")
    print(f"Ruido e2 en cifrado con nivel {error_level}: {e2}")
    u = reduce_modulus(a * e1, q, n)
    v = reduce_modulus(b * e1 + e2 + Polynomial(m), q, n)
    return u, v

def rlwe_decrypt(ciphertext, s, q, n):
    """
    Descifra un mensaje cifrado en RLWE.

    Parámetros:
    - ciphertext: Tupla (u, v) que representa el mensaje cifrado.
    - s: Clave secreta utilizada para descifrar.
    - q: Módulo del polinomio.
    - n: Grado del polinomio.

    Retorna:
    - Mensaje descifrado (vector de coeficientes).
    """
    u, v = ciphertext
    decrypted_poly = reduce_modulus(v - s * u, q, n)
    print(f"Valor de v - s*u antes de reducir módulo: {v - s * u}")
    recovered_message = np.mod(np.round(decrypted_poly.coef).astype(int), q)
    return recovered_message

---
A continuación podemos probar el problema RLWE y jugar con posibles combinaciones de sus parámetros. Puede cambiar el grado del polinomio, el número primo e incluso el error. El error lo hemos divido en 10 posibles niveles, donde: 
- 1: Mínimo ruido, casi sin efecto.
- 2-3: Ligero ruido, debería permitir recuperación exacta.
- 4-5: Ruido moderado, puede empezar a causar ligeros errores.
- 6-7: Ruido notable, la recuperación podría no ser exacta.
- 8-9: Ruido alto, posible corrupción de mensaje.
- 10: Ruido máximo, alta probabilidad de error en descifrado.

Obsérvse que cuanto más pequeño es q y más grande es el ruido, más se acumula el error y afecta al mensaje.

Además, el grado del polinomio n también influye en la seguridad y la complejidad del esquema. Un n pequeño hace que las operaciones sean más rápidas, pero puede comprometer la seguridad, mientras que un n grande aumenta la robustez criptográfica a costa de un mayor costo computacional.

El parámetro q, que define el módulo de los coeficientes, también juega un papel crucial. Un q mayor permite una mayor tolerancia al ruido, reduciendo la probabilidad de errores en el descifrado, pero puede hacer que los cálculos sean más costosos.

In [5]:
# Parámetros
n = 3
q = 23
error_level = 1  # Controla el nivel de error en todo el proceso (1-10)

a, b, secret_key = generate_rlwe_instance(n, q, error_level)
mensaje = np.random.randint(0, q, n)

encrypted_message = rlwe_encrypt(mensaje, a, b, q, n, error_level)
decrypted_message = rlwe_decrypt(encrypted_message, secret_key, q, n)

# Mostrar resultados con formato mejorado
print("\n🧩==============================")
print("       🔑 INSTANCIA RLWE        ")
print("==============================")
print(f"\n🔐 Polinomio 'a' (público): {a}")
print(f"🔐 Polinomio 'b' (público): {b}")
print(f"🛡️ Clave secreta 's' (privada): {secret_key}")
print("\n📜==============================")
print("       ✉️ MENSAJE ORIGINAL      ")
print("==============================")
print(f"📝 Mensaje: {mensaje}")
print("\n🔒==============================")
print("        📨 MENSAJE CIFRADO      ")
print("==============================")
print(f"🔑 Componente 'u': {encrypted_message[0]}")
print(f"🔑 Componente 'v': {encrypted_message[1]}")
print("\n🔓==============================")
print("      🔑 MENSAJE DESCIFRADO     ")
print("==============================")
print(f"✅ Mensaje recuperado: {decrypted_message}")

Error generado con nivel 1 (σ=0.2): 0.0 + 0.0·x + 0.0·x²
Ruido e en RLWE instancia con nivel 1: 0.0 + 0.0·x + 0.0·x²
Error generado con nivel 1 (σ=0.2): 0.0 + 0.0·x + 0.0·x²
Error generado con nivel 1 (σ=0.2): 0.0 + 0.0·x + 0.0·x²
Ruido e1 en cifrado con nivel 1: 0.0 + 0.0·x + 0.0·x²
Ruido e2 en cifrado con nivel 1: 0.0 + 0.0·x + 0.0·x²
Valor de v - s*u antes de reducir módulo: 6.0 + 20.0·x + 10.0·x²

🧩==============================
       🔑 INSTANCIA RLWE        

🔐 Polinomio 'a' (público): 8.0 + 2.0·x + 21.0·x²
🔐 Polinomio 'b' (público): 18.0 + 7.0·x + 6.0·x²
🛡️ Clave secreta 's' (privada): 8.0 + 19.0·x + 21.0·x²

📜==============================
       ✉️ MENSAJE ORIGINAL      
📝 Mensaje: [ 6 20 10]

🔒==============================
        📨 MENSAJE CIFRADO      
🔑 Componente 'u': 0.0
🔑 Componente 'v': 6.0 + 20.0·x + 10.0·x²

🔓==============================
      🔑 MENSAJE DESCIFRADO     
✅ Mensaje recuperado: [ 6 20 10]


---
# 4. MLWE


El problema de Learning With Errors (MLWE, por sus siglas en inglés) es una variante del problema RLWE, pero aplicado a matrices en lugar de polinomios. Al igual que en RLWE, el MLWE se basa en la dificultad de resolver sistemas de ecuaciones en un espacio matricial, con un pequeño error añadido a las soluciones. Este problema también se considera seguro frente a ataques cuánticos, lo que lo hace interesante para la criptografía post-cuántica.

En el caso específico de MLWE, el objetivo es encontrar una clave secreta $S$, dada una instancia pública que consiste en dos matrices $A$ y $B$, que están relacionadas de la siguiente manera:

$$
B = A \cdot S + E
$$

donde $E$ es un pequeño error aleatorio y $S$ es la clave secreta.

La dificultad del problema radica en que es computacionalmente difícil obtener $S$ de manera eficiente a partir de $A$ y $B$, debido a la presencia del error $E$.

<br>

El código implementa un esquema de cifrado basado en el problema MLWE. Primero, genera una instancia del problema creando dos matrices públicas $A$ y $B$, y una clave secreta $S$, que están relacionadas por la ecuación $B = A \cdot S + E$, donde $E$ es un pequeño error aleatorio. Luego, utiliza esta instancia para cifrar y descifrar mensajes. El cifrado se realiza mediante la generación de dos componentes $u$ y $v$, que dependen de errores aleatorios y del mensaje. El descifrado recupera el mensaje original utilizando la clave secreta $S$ para despejar la información del cifrado. El proceso involucra operaciones de multiplicación matricial y reducción módulo $q$ en el espacio de matrices $Z_q^{n \times n}$.



In [11]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix
    
    def __repr__(self):
        return f"Matrix({self.matrix})"

def generate_secret_key(n, q):
    """
    Genera una clave secreta como una matriz en Z_q^(n x n).

    Parámetros:
    - n: Dimensión de la matriz.
    - q: Módulo del polinomio.

    Retorna:
    - secret_key: Matriz secreta generada.
    """
    return Matrix(np.random.randint(0, q, (n, n)) % q)

def generate_error(n, error_level, q):
    """
    Genera un error pequeño basado en el nivel de ruido controlado.

    Parámetros:
    - n: Dimensión de la matriz.
    - error_level: Nivel de error (1-10).
    - q: Módulo de la matriz.

    Retorna:
    - error: Matriz de error generada.
    """
    scaled_error = error_level / 5  # Escala el ruido de 0.2 a 2
    error = Matrix(np.round(np.random.normal(0, scaled_error, (n, n))).astype(int) % q)
    print(f"\n⚠️ Error generado con nivel {error_level} (σ={scaled_error}):\n{error}\n")
    return error

def generate_mlwe_instance(n, q, error_level):
    """
    Genera una instancia del problema MLWE: (A, B = A * S + E) en Z_q^(n x n).

    Parámetros:
    - n: Dimensión de la matriz.
    - q: Módulo de la matriz.
    - error_level: Nivel de error (1-10).

    Retorna:
    - A: Matriz pública 'A'.
    - B: Matriz pública 'B'.
    - secret_key: Clave secreta 'S'.
    """
    A = Matrix(np.random.randint(0, q, (n, n)) % q)
    S = generate_secret_key(n, q)
    E = generate_error(n, error_level, q)
    B = Matrix(np.mod(A.matrix @ S.matrix + E.matrix, q))
    print(f"🔹 Ruido E en MLWE instancia con nivel {error_level}:\n{E}\n")
    return A, B, S

def mlwe_encrypt(m, A, B, q, n, error_level):
    """
    Cifra un mensaje m en Z_q^(n x n) usando la instancia MLWE.

    Parámetros:
    - m: Mensaje a cifrar (matriz de coeficientes).
    - A: Matriz pública 'A'.
    - B: Matriz pública 'B'.
    - q: Módulo de la matriz.
    - n: Dimensión de la matriz.
    - error_level: Nivel de error (1-10).

    Retorna:
    - u: Componente 'u' del cifrado.
    - v: Componente 'v' del cifrado.
    """
    e1 = generate_error(n, error_level, q)
    e2 = generate_error(n, error_level, q)
    print(f"🔹 Ruido e1 en cifrado con nivel {error_level}:\n{e1}\n")
    print(f"🔹 Ruido e2 en cifrado con nivel {error_level}:\n{e2}\n")
    u = Matrix(np.mod(A.matrix @ e1.matrix, q))
    v = Matrix(np.mod(B.matrix @ e1.matrix + e2.matrix + m, q))
    return u, v

def mlwe_decrypt(ciphertext, S, q, n):
    """
    Descifra un mensaje cifrado en MLWE.

    Parámetros:
    - ciphertext: Tupla (u, v) que representa el mensaje cifrado.
    - S: Clave secreta utilizada para descifrar.
    - q: Módulo de la matriz.
    - n: Dimensión de la matriz.

    Retorna:
    - m_rec: Mensaje descifrado.
    """
    u, v = ciphertext
    m_rec = Matrix(np.mod(v.matrix - S.matrix @ u.matrix, q))
    return m_rec.matrix.astype(int)

---
A continuación podemos probar el problema MLWE y jugar con posibles combinaciones de sus parámetros. Puede cambiar la dimension de la matriz, el número primo e incluso el error. De igual manera que la implementación de MLWE, el error lo hemos divido en 10 posibles niveles, donde: 

- 1: Mínimo ruido, casi sin efecto.
- 2-3: Ligero ruido, debería permitir recuperación exacta.
- 4-5: Ruido moderado, puede empezar a causar ligeros errores.
- 6-7: Ruido notable, la recuperación podría no ser exacta.
- 8-9: Ruido alto, posible corrupción de mensaje.
- 10: Ruido máximo, alta probabilidad de error en descifrado.

In [12]:
# Parámetros
n = 2  # Dimensión de la matriz (tamaño del problema MLWE)
q = 23  # Módulo primo pequeño
error_level = 1  # Nivel de error en todo el proceso (1-10)

# Generar instancia MLWE
A, B, secret_key = generate_mlwe_instance(n, q, error_level)

# Presentar instancia MLWE
print("\n🧩==============================")
print("       🔑 INSTANCIA MLWE        ")
print("==============================")
print(f"\n🔐 Matriz 'A' (pública):")
print(A)
print(f"\n🔐 Matriz 'B' (pública):")
print(B)
print(f"\n🛡️ Clave secreta 'S' (privada):")
print(secret_key)

# Generar mensaje a cifrar
mensaje = np.random.randint(0, q, (n, n)) % q
print("\n📜==============================")
print("       ✉️ MENSAJE ORIGINAL      ")
print("==============================")
print(f"📝 Mensaje:")
print(mensaje)

# Cifrado y descifrado
encrypted_message = mlwe_encrypt(mensaje, A, B, q, n, error_level)
print("\n🔒==============================")
print("        📨 MENSAJE CIFRADO      ")
print("==============================")
print(f"🔑 Componente 'u' (cifrado):")
print(encrypted_message[0])
print(f"🔑 Componente 'v' (cifrado):")
print(encrypted_message[1])

decrypted_message = mlwe_decrypt(encrypted_message, secret_key, q, n)
print("\n🔓==============================")
print("      🔑 MENSAJE DESCIFRADO     ")
print("==============================")
print(f"✅ Mensaje recuperado:")
print(decrypted_message)


⚠️ Error generado con nivel 1 (σ=0.2):
Matrix([[0 0]
 [0 0]])

🔹 Ruido E en MLWE instancia con nivel 1:
Matrix([[0 0]
 [0 0]])


🧩==============================
       🔑 INSTANCIA MLWE        

🔐 Matriz 'A' (pública):
Matrix([[ 0  5]
 [18 12]])

🔐 Matriz 'B' (pública):
Matrix([[ 6 18]
 [19 19]])

🛡️ Clave secreta 'S' (privada):
Matrix([[ 0  3]
 [15 22]])

📜==============================
       ✉️ MENSAJE ORIGINAL      
📝 Mensaje:
[[ 8 20]
 [16 19]]

⚠️ Error generado con nivel 1 (σ=0.2):
Matrix([[0 0]
 [0 0]])


⚠️ Error generado con nivel 1 (σ=0.2):
Matrix([[0 0]
 [0 0]])

🔹 Ruido e1 en cifrado con nivel 1:
Matrix([[0 0]
 [0 0]])

🔹 Ruido e2 en cifrado con nivel 1:
Matrix([[0 0]
 [0 0]])


🔒==============================
        📨 MENSAJE CIFRADO      
🔑 Componente 'u' (cifrado):
Matrix([[0 0]
 [0 0]])
🔑 Componente 'v' (cifrado):
Matrix([[ 8 20]
 [16 19]])

🔓==============================
      🔑 MENSAJE DESCIFRADO     
✅ Mensaje recuperado:
[[ 8 20]
 [16 19]]
